In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import pydot
from sklearn.covariance import GraphicalLasso
from PIL import Image
from IPython.display import display
import japanize_matplotlib
import os

# ==========================================
# 設定セクション
# ==========================================

# 入出力ファイルの設定
# 前のステップで作成したポリコリック相関行列のファイル名
INPUT_FILE_NAME = 'correlation_matrix.csv' 
OUTPUT_GRAPH_IMAGE = 'network_graph.png'

# フォント設定
plt.rcParams['font.size'] = 12

# カラム名の英語変換マッピング
# Graphvizは日本語で文字化けしやすいため、ノード名は英語にするのが無難
rename_dict = {
    # 属性（分析から除外する場合は下記リストで指定）
    "年齢を教えてください。": "Age",
    "年齢": "Age",
    
    # 味覚項目
    "甘味をどの程度感じましたか？（１／５）": "Sweetness",
    "甘味": "Sweetness",
    "うま味をどの程度感じましたか？（２／５）": "Umami",
    "うま味": "Umami",
    "苦味をどの程度感じましたか？（３／５）": "Bitterness",
    "苦味": "Bitterness",
    "酸味をどの程度感じましたか？（４／５）": "Sourness",
    "酸味": "Sourness",
    "塩味をどの程度感じましたか？（５／５）": "Saltiness",
    "塩味": "Saltiness",
    
    # 視覚項目
    "華やかさをどの程度感じましたか？（１／５）": "Floral",
    "華やかさ": "Floral",
    "開放感をどの程度感じましたか？（２／５）": "Openness",
    "開放感": "Openness",
    "爽やかさをどの程度感じましたか？（３／５）": "Freshness",
    "爽やかさ": "Freshness",
    "賑い感をどの程度感じましたか？（４／５）": "Liveliness",
    "賑わい感をどの程度感じましたか？（４／５）": "Liveliness",
    "賑わい感": "Liveliness",
    "高揚感をどの程度感じましたか？（５／５）": "Excitement",
    "高揚感": "Excitement",
    
    # 目的変数
    "クラフトビール「白蒲田」をまた飲んでみたいと思いますか？": "Drink Again",
    "リピート意向": "Drink Again"
}

# 分析から除外する項目（ネットワーク図に含めない変数）
drop_columns = ["Age"] 

def partial_correlation(precision_matrix):
    """
    精度行列（Precision Matrix）を偏相関行列（Partial Correlation Matrix）に変換する
    """
    D = np.sqrt(np.diag(precision_matrix)) # 各変数の標準偏差に対応する項
    partial_corr = -(precision_matrix / np.outer(D, D)) + np.eye(len(D)) * 2
    # 対角成分は1.0にするための補正（上記計算で対角は正しくなるが念のため）
    np.fill_diagonal(partial_corr, 1.0)
    return partial_corr

def visualize_graph_with_pydot(partial_corr_matrix, nodes, threshold=0.05, output_file="graph.png"):
    """
    pydot (Graphviz) を用いてネットワーク図を描画・保存する
    """
    g = pydot.Dot(graph_type="graph", layout="dot", rankdir="LR", overlap="false", splines="true")

    # ノードの作成
    node_objects = {}
    for node in nodes:
        node_obj = pydot.Node(
            node, 
            style="filled", 
            fillcolor="white", 
            shape="ellipse",
            fontname="Arial", 
            fontsize="12", 
            width="1.5", 
            height="0.6"
        )
        g.add_node(node_obj)
        node_objects[node] = node_obj

    # エッジの太さ調整用
    # 行列内の最大値・最小値を取得（しきい値を超えたものの中で）
    valid_weights = np.abs(partial_corr_matrix)[np.abs(partial_corr_matrix) > threshold]
    if len(valid_weights) > 0:
        min_weight = np.min(valid_weights)
        max_weight = np.max(valid_weights)
    else:
        min_weight, max_weight = 0, 1

    # エッジの追加
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            weight = partial_corr_matrix[i, j]
            if abs(weight) > threshold:
                # 線の太さを相関の強さに応じて調整
                if max_weight > min_weight:
                    scaled_penwidth = (abs(weight) - min_weight) / (max_weight - min_weight) * 3 + 0.5
                else:
                    scaled_penwidth = 1.0
                
                # 正の相関は実線、負の相関は点線にするなどの装飾も可能
                style = "solid" if weight > 0 else "dashed"
                color = "black" if weight > 0 else "red"
                
                edge = pydot.Edge(
                    node_objects[nodes[i]], 
                    node_objects[nodes[j]], 
                    penwidth=scaled_penwidth,
                    style=style,
                    color=color
                )
                # エッジのラベルに数値を表示
                edge.set_label(f"{weight:.2f}") 
                g.add_edge(edge)

    # ノードの色付け（接続数＝次数に応じて濃淡をつける）
    # 行列の非ゼロ要素（しきい値以上）をカウント
    num_edges = np.sum(np.abs(partial_corr_matrix) > threshold, axis=0)
    max_num_edges = max(np.max(num_edges), 1)
    
    for i, node in enumerate(nodes):
        # 青色の濃淡で重要度を表現 (0.0 〜 1.0)
        intensity = num_edges[i] / max_num_edges * 0.5  # 濃すぎないように調整
        fillcolor = f"0.0 {intensity} 1.0" # HSV色空間など、pydotの仕様に合わせる（ここでは青固定で彩度調整のイメージ）
        # 簡易的に水色～青にするなら色名指定やHexコード推奨
        # ここではシンプルに白背景のままにするか、接続が多いものを強調
        if num_edges[i] > 0:
            node_objects[node].set_style("filled")
            node_objects[node].set_fillcolor("lightblue")

    # ファイル保存
    try:
        g.write_png(output_file)
        print(f"ネットワーク図を保存しました: {output_file}")
        
        # 画像をNotebook上に表示
        image = Image.open(output_file)
        display(image)
    except Exception as e:
        print(f"画像保存エラー: {e}")
        print("Graphvizがインストールされていない可能性があります。")


# ==========================================
# メイン処理
# ==========================================

# データの読み込み
try:
    # index_col=0 で行名（変数名）を正しく読み込む
    correlation_matrix = pd.read_csv(INPUT_FILE_NAME, index_col=0)
    print(f"データ読み込み成功: {INPUT_FILE_NAME}")
except FileNotFoundError:
    print(f"エラー: ファイル '{INPUT_FILE_NAME}' が見つかりません。")
    # ダミー処理用（実行ストップ）
    correlation_matrix = None

if correlation_matrix is not None:
    # 変数名のリネームと整理
    # 行と列の両方をリネーム
    correlation_matrix.rename(columns=rename_dict, index=rename_dict, inplace=True)
    
    # 不要なカラム（Ageなど）の削除
    # 存在するものだけ削除
    drop_targets = [col for col in drop_columns if col in correlation_matrix.columns]
    cov = correlation_matrix.drop(index=drop_targets, columns=drop_targets)
    
    print("分析対象の変数:")
    print(cov.columns.tolist())

    # ベイズ最適化によるハイパーパラメータ探索 (Optuna)
    print("\nOptunaによる最適化を開始します...")
    
    def bayes_objective(trial):
        # 探索する正則化パラメータ alpha (L1ペナルティ)
        alpha = trial.suggest_float("alpha", 0.001, 1.0, log=True)
        
        try:
            model = GraphicalLasso(alpha=alpha, max_iter=200, tol=1e-4)
            model.fit(cov)
            
            # 精度行列 (Precision Matrix)
            omega = model.precision_
            
            # EBIC (拡張ベイズ情報量基準) の計算
            # 尤度、自由度（エッジ数）、サンプルサイズからモデルの良さを評価
            n_features = len(cov.columns)
            n_samples = 100 # サンプルサイズ（本来は元データの行数を入れるべきだが、相関行列だけの場合は仮定値または定数として扱う）
            
            # 非ゼロエッジの数 (対角成分を除く)
            n_edges = (np.sum(omega != 0) - n_features) / 2
            
            # 対数尤度の近似計算（Graphical Lassoの目的関数の一部を利用）
            # ここでは厳密な尤度計算よりも、モデルのスパース性と適合度のバランスを見る
            mle = n_samples * (np.log(np.linalg.det(omega)) - np.trace(np.dot(cov, omega)))
            
            # EBICの計算式 (gamma=0.5 とすることが多い)
            gamma = 0.5
            ebic = -mle + n_edges * np.log(n_samples) + 4 * n_edges * gamma * np.log(n_features)
            
            return ebic
        except Exception:
            return float('inf')

    # 最適化の実行
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(bayes_objective, n_trials=50) # トライアル数は適宜調整

    best_alpha = study.best_params["alpha"]
    print(f"最適な alpha (λ): {best_alpha:.4f}")

    # 最適パラメータでのモデル推定
    best_model = GraphicalLasso(alpha=best_alpha, max_iter=500, tol=1e-4)
    best_model.fit(cov)
    precision_matrix = best_model.precision_

    # 偏相関行列への変換
    partial_corr_matrix = partial_correlation(precision_matrix)
    
    # DataFrame形式にして確認しやすくする
    partial_corr_df = pd.DataFrame(
        partial_corr_matrix, 
        index=cov.columns, 
        columns=cov.columns
    )
    
    # ネットワーク図の描画
    # しきい値（これ以下の偏相関は線を引かない）の設定
    THRESHOLD = 0.03 
    
    nodes = cov.columns
    visualize_graph_with_pydot(
        partial_corr_matrix, 
        nodes, 
        threshold=THRESHOLD, 
        output_file=OUTPUT_GRAPH_IMAGE
    )